[View in Colaboratory](https://colab.research.google.com/github/mdasadul/bangla_embedding/blob/master/Word2Vec_bengali.ipynb)

In [0]:
# -*- coding: utf-8 -*-

from __future__ import unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.font_manager as fm


prop = fm.FontProperties(fname='kalpurush.ttf')


corpus_raw = """এল.রবিন্সের সংজ্ঞাটি বিশ্লেষণ করলে দেখা যায় যে এটি মানব জীবনের তিনটি মৌলিক বৈশিষ্টের উপর প্রতিষ্ঠিত যথা অসীম অভাব সীমিত সম্পদ ও বিকল্প ব্যাবহারযোগ্য সম্পদ ।
ইকোনোমেট্রিক্স অর্থনৈতিক মডেলসমূহের উপাত্ত সম্পর্কিত বিশ্লেষনে গাণিতিক ও পরিসংখ্যানিক পদ্ধতিতে প্রয়োগ করা হয় ।
প্রান্তিক অর্থনৈতিক তত্ত্বে আয় ও সম্পদকে প্রতিবন্ধক বিষয় ধরে ভোক্তা অধিক পছন্দ অবস্থানে পৌঁছতে চেষ্টা করে তা নিয়ে বিশ্লেষন করা হয় ।
ভোক্তা বা অন্য প্রতিনিধির ক্ষেত্রে যদি সম্পদের পূর্ন ব্যবহার করা হয় উৎপাদন সম্ভাবনা ও দূস্প্রাপ্যতার ব্যবধান কমে আসবে যেমন আয়ের ক্ষেত্রে বেকার সময় সরকারী পন্যের জন্য বেসরকারী পন্য ভবিষ্যৎ ভোগের জন্য বর্তমান ভোগ ।
ইহা সাধারণত ক্যামব্রিজ বিশ্ববিদ্যালয় ও জন রবিনসনের কাজের সাথে সম্পর্কযুক্ত ।
কল্যান সংজ্ঞা
অনেক অর্থনীতিবিদ “ অনুমিত শর্ত ” থেকে পর্যবেক্ষণ মুলকে পরিবর্তিত হওয়া ধারণার উদাহরন দিয়েছেন তা নিম্নরুপঃ
জন স্টুয়ার্ট মিল বা লিওন ওয়ালার্স এর মত কিছু অর্থনীতিবিদ উল্লেখ করেন যে সম্পদ উৎপাদনকে ইহার বন্টনের সহিৎ আবদ্ধ করা যায়না ।
উদাহরন হিসেবে বলা যায় যে স্বত্বাধিকারী পদ্ধতি সমঅধিকার পদ্ধতি বা সমাজের উচ্চ শ্রেণীর পদ্ধতির অর্থনীতিতে তাদের যোগ্যতা বাচ-বিচার ছাড়াই প্রয়োগ করা হয় ।
উইকিপিডিয়ার বাংলা সংস্করণে স্বাগতম ।
অর্থাৎ এই প্রক্রিয়ায় সার্ভারের সাথে যোগাযোগের জন্য ব্যবহারকারীর কোন তথ্য পোস্ট করা পর্যন্ত অপেক্ষা করতে হয় না বরং বর্তমান পাতাটি ব্যবহারকারী কর্তৃক পর্যবেক্ষণ বা কোন ফর্ম পূরণের সময়ই পটভূমিতে সার্ভারের সাথে তথ্য আদান-প্রদান চালু থাকে ।
ফায়ারফক্স একটি মুক্ত সোর্স ওয়েব ব্রাউজার ।
অন্যান্য অনেক ব্রাউজারে এসব সুবিধার কিছু কিছু গ্রহণ করা হয়েছিল কিন্তু ফায়ারফক্সই প্রথম ব্রাউজার যাতে সবগুলো সুবিধা যুক্ত হয় ।
এগুলোর মধ্যে রয়েছে ডম ইন্সপেক্টর বা এক্সটেনশন যেমন : ফায়ারবাগ ।
জাপানের পরাজয়ের মাধ্যমে যুদ্ধের সমাপ্তি ঘটে এবং এতে দেশের বেশির ভাগ শিল্পকারখানা পরিবহন নেটওয়ার্ক এবং নাগরিক অবকাঠামো ধ্বংস হয়ে যায় ।"""
#print(corpus_raw)
# convert to lower case
#corpus_raw = corpus_raw.encode('utf-8')
#corpus_raw = corpus_raw.lower()

words = []
for line in corpus_raw.splitlines():
    for word in line.split():
        words.append(word)
        print(word)

words = set(words) # so that all duplicate words are removed
word2int = {}
int2word = {}
vocab_size = len(words) # gives the total number of unique words

for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

# raw sentences is a list of sentences.
raw_sentences = corpus_raw.splitlines()
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())

WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

x_train = [] # input word
y_train = [] # output word

for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))

# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

# making placeholders for x_train and y_train
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

EMBEDDING_DIM = 10 # you can choose your own number
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))


sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))

# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)

n_iters = 1000
# train for n_iter iterations

for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

vectors = sess.run(W1 + b1)

def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index


from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
vectors = model.fit_transform(vectors) 

from sklearn import preprocessing

normalizer = preprocessing.Normalizer()
vectors =  normalizer.fit_transform(vectors, 'l2')

print(vectors)
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt


fig, ax = plt.subplots()
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
print(words)
for word in words:
    print(word, vectors[word2int[word]][1])
    ax.annotate(word, (vectors[word2int[word]][0],vectors[word2int[word]][1] ),fontproperties=prop)
plt.show()

এল.রবিন্সের
সংজ্ঞাটি
বিশ্লেষণ
করলে
দেখা
যায়
যে
এটি
মানব
জীবনের
তিনটি
মৌলিক
বৈশিষ্টের
উপর
প্রতিষ্ঠিত
যথা
অসীম
অভাব
সীমিত
সম্পদ
ও
বিকল্প
ব্যাবহারযোগ্য
সম্পদ
।
ইকোনোমেট্রিক্স
অর্থনৈতিক
মডেলসমূহের
উপাত্ত
সম্পর্কিত
বিশ্লেষনে
গাণিতিক
ও
পরিসংখ্যানিক
পদ্ধতিতে
প্রয়োগ
করা
হয়
।
প্রান্তিক
অর্থনৈতিক
তত্ত্বে
আয়
ও
সম্পদকে
প্রতিবন্ধক
বিষয়
ধরে
ভোক্তা
অধিক
পছন্দ
অবস্থানে
পৌঁছতে
চেষ্টা
করে
তা
নিয়ে
বিশ্লেষন
করা
হয়
।
ভোক্তা
বা
অন্য
প্রতিনিধির
ক্ষেত্রে
যদি
সম্পদের
পূর্ন
ব্যবহার
করা
হয়
উৎপাদন
সম্ভাবনা
ও
দূস্প্রাপ্যতার
ব্যবধান
কমে
আসবে
যেমন
আয়ের
ক্ষেত্রে
বেকার
সময়
সরকারী
পন্যের
জন্য
বেসরকারী
পন্য
ভবিষ্যৎ
ভোগের
জন্য
বর্তমান
ভোগ
।
ইহা
সাধারণত
ক্যামব্রিজ
বিশ্ববিদ্যালয়
ও
জন
রবিনসনের
কাজের
সাথে
সম্পর্কযুক্ত
।
কল্যান
সংজ্ঞা
অনেক
অর্থনীতিবিদ
“
অনুমিত
শর্ত
”
থেকে
পর্যবেক্ষণ
মুলকে
পরিবর্তিত
হওয়া
ধারণার
উদাহরন
দিয়েছেন
তা
নিম্নরুপঃ
জন
স্টুয়ার্ট
মিল
বা
লিওন
ওয়ালার্স
এর
মত
কিছু
অর্থনীতিবিদ
উল্লেখ
করেন
যে
সম্পদ
উৎপাদনকে
ইহার
বন্টনের
সহিৎ
আবদ্ধ
করা
যায়না
।
উদাহরন
হিসেবে
বলা
যায়
যে
স্বত্বাধিকারী
পদ্ধতি
সমঅধি

(u'loss is : ', 8.100178)
(u'loss is : ', 8.094299)
(u'loss is : ', 8.088435)
(u'loss is : ', 8.082585)
(u'loss is : ', 8.076753)
(u'loss is : ', 8.070935)
(u'loss is : ', 8.065132)
(u'loss is : ', 8.059344)
(u'loss is : ', 8.053571)
(u'loss is : ', 8.047813)
(u'loss is : ', 8.04207)
(u'loss is : ', 8.036343)
(u'loss is : ', 8.030629)
(u'loss is : ', 8.024932)
(u'loss is : ', 8.019247)
(u'loss is : ', 8.013577)
(u'loss is : ', 8.007921)
(u'loss is : ', 8.002281)
(u'loss is : ', 7.996655)
(u'loss is : ', 7.991044)
(u'loss is : ', 7.985444)
(u'loss is : ', 7.9798613)
(u'loss is : ', 7.9742928)
(u'loss is : ', 7.968735)
(u'loss is : ', 7.9631934)
(u'loss is : ', 7.9576645)
(u'loss is : ', 7.9521503)
(u'loss is : ', 7.9466505)
(u'loss is : ', 7.9411635)
(u'loss is : ', 7.9356885)
(u'loss is : ', 7.930229)
(u'loss is : ', 7.9247823)
(u'loss is : ', 7.9193487)
(u'loss is : ', 7.9139285)
(u'loss is : ', 7.908522)
(u'loss is : ', 7.903127)
(u'loss is : ', 7.897748)
(u'loss is : ', 7.8923793)
(

(u'loss is : ', 6.9707584)
(u'loss is : ', 6.967269)
(u'loss is : ', 6.963785)
(u'loss is : ', 6.960307)
(u'loss is : ', 6.9568343)
(u'loss is : ', 6.9533687)
(u'loss is : ', 6.9499083)
(u'loss is : ', 6.946454)
(u'loss is : ', 6.9430056)
(u'loss is : ', 6.939562)
(u'loss is : ', 6.936126)
(u'loss is : ', 6.9326944)
(u'loss is : ', 6.9292684)
(u'loss is : ', 6.925849)
(u'loss is : ', 6.922436)
(u'loss is : ', 6.919028)
(u'loss is : ', 6.9156246)
(u'loss is : ', 6.912228)
(u'loss is : ', 6.908838)
(u'loss is : ', 6.9054523)
(u'loss is : ', 6.902072)
(u'loss is : ', 6.8986974)
(u'loss is : ', 6.895329)
(u'loss is : ', 6.8919663)
(u'loss is : ', 6.888609)
(u'loss is : ', 6.885257)
(u'loss is : ', 6.8819118)
(u'loss is : ', 6.87857)
(u'loss is : ', 6.8752346)
(u'loss is : ', 6.8719063)
(u'loss is : ', 6.868582)
(u'loss is : ', 6.865264)
(u'loss is : ', 6.8619514)
(u'loss is : ', 6.8586426)
(u'loss is : ', 6.855341)
(u'loss is : ', 6.852043)
(u'loss is : ', 6.848752)
(u'loss is : ', 6.84546

(u'loss is : ', 6.244412)
(u'loss is : ', 6.2420316)
(u'loss is : ', 6.2396545)
(u'loss is : ', 6.2372804)
(u'loss is : ', 6.2349095)
(u'loss is : ', 6.232542)
(u'loss is : ', 6.2301774)
(u'loss is : ', 6.2278147)
(u'loss is : ', 6.2254577)
(u'loss is : ', 6.2231016)
(u'loss is : ', 6.2207503)
(u'loss is : ', 6.2184024)
(u'loss is : ', 6.216057)
(u'loss is : ', 6.2137146)
(u'loss is : ', 6.211376)
(u'loss is : ', 6.20904)
(u'loss is : ', 6.2067075)
(u'loss is : ', 6.204378)
(u'loss is : ', 6.202052)
(u'loss is : ', 6.199729)
(u'loss is : ', 6.1974087)
(u'loss is : ', 6.1950917)
(u'loss is : ', 6.192778)
(u'loss is : ', 6.1904683)
(u'loss is : ', 6.18816)
(u'loss is : ', 6.185856)
(u'loss is : ', 6.1835546)
(u'loss is : ', 6.1812572)
(u'loss is : ', 6.1789618)
(u'loss is : ', 6.1766696)
(u'loss is : ', 6.17438)
(u'loss is : ', 6.1720943)
(u'loss is : ', 6.1698127)
(u'loss is : ', 6.167532)
(u'loss is : ', 6.165255)
(u'loss is : ', 6.1629825)
(u'loss is : ', 6.160712)
(u'loss is : ', 6.1

(u'loss is : ', 5.7323046)
(u'loss is : ', 5.730571)
(u'loss is : ', 5.728839)
(u'loss is : ', 5.727109)
(u'loss is : ', 5.7253814)
(u'loss is : ', 5.723655)
(u'loss is : ', 5.7219315)
(u'loss is : ', 5.72021)
(u'loss is : ', 5.7184896)
(u'loss is : ', 5.7167706)
(u'loss is : ', 5.715054)
(u'loss is : ', 5.7133403)
(u'loss is : ', 5.711628)
(u'loss is : ', 5.709917)
(u'loss is : ', 5.7082095)
(u'loss is : ', 5.7065034)
(u'loss is : ', 5.7047987)
(u'loss is : ', 5.7030945)
(u'loss is : ', 5.701395)
(u'loss is : ', 5.6996965)
(u'loss is : ', 5.698)
(u'loss is : ', 5.696305)
(u'loss is : ', 5.694612)
(u'loss is : ', 5.692921)
(u'loss is : ', 5.691232)
(u'loss is : ', 5.689545)
(u'loss is : ', 5.687859)
(u'loss is : ', 5.686176)
(u'loss is : ', 5.684494)
(u'loss is : ', 5.682815)
(u'loss is : ', 5.681137)
(u'loss is : ', 5.6794624)
(u'loss is : ', 5.6777873)
(u'loss is : ', 5.676115)
(u'loss is : ', 5.674444)
(u'loss is : ', 5.672777)
(u'loss is : ', 5.671111)
(u'loss is : ', 5.6694455)
(u

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:138: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catc

IOError: ignored